In [23]:
import pandas as pd

In [24]:
csv = pd.read_csv("../combined_profile.csv",keep_default_na=True,na_values=["None", "none", "NaN", "-"],low_memory=False,dtype={"company_id": str})
csv["company_id"] = csv["company_id"].astype(str)

In [25]:
df = csv
bad_ids = df[~df["company_id"].str.fullmatch(r"\d{4}")]

In [26]:
# Ensure company_id is string
csv["company_id"] = csv["company_id"].astype(str)

# Detect bad IDs (not exactly 4 digits)
bad_mask = ~csv["company_id"].str.fullmatch(r"\d{4}")

# Fix only the bad ones
csv.loc[bad_mask, "company_id"] = csv.loc[bad_mask, "company_id"].str.zfill(4)

In [27]:
df = csv

In [28]:
merged = df

In [29]:
bursa_registration = pd.read_csv("../matched_companies_from_ssm.csv",dtype={"company_id": str,"companyNo": str})
bursa_registration["companyNo"] = (
    bursa_registration["companyNo"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .str.strip()
)



In [30]:
company_id = pd.read_csv("../../../list_bursa_ids/bursa_company_list.csv", dtype={"company_id": str})
company_id["company_id"] = company_id["company_id"].str.strip().str.zfill(4)


In [31]:
merge = pd.merge(bursa_registration,company_id,on="company_name",how="inner")

In [32]:
merge = merge.drop(columns="company_type")
merge = merge.rename(columns={
    "company_name": "company_name_bursa",
    "name_db": "company_name_api",
    "companyNo": "registration_number",
    "oldCompanyNo": "old_registration_number"
})

In [33]:
merged = df

In [34]:
merge["company_id"] = merge["company_id"].str.strip()
merged["company_id"] = merged["company_id"].str.strip()

In [35]:
merge["company_id"] = merge["company_id"].astype(str)
merged["company_id"] = merged["company_id"].astype(str)
final_merge = merge.merge(merged, on="company_id", how="inner")

In [36]:
market_cap = pd.read_csv("../market_info_sample.csv",dtype={"company_id": str})
market_cap = market_cap[['company_id','market_cap_mil']]


In [37]:
final_merge = pd.merge(final_merge,market_cap, on="company_id", how="left")

In [38]:
df=final_merge

In [39]:
# ── Your DataFrame ──
df = df.copy()
df.columns = df.columns.str.replace("-", " ").str.strip().str.strip().str.lower().str.replace(" ", "_").str.replace("-", "_")

In [40]:
import re
df.columns = (
    df.columns
    .str.replace("-", " ", regex=False)        # Replace hyphens with spaces
    .str.replace("/", " ", regex=False) 
    .str.strip()
    .str.lower()
    .str.replace(r"[^\w\s]", "", regex=True)   # Remove non-word characters like . , &
    .str.replace(r"\s+", "_", regex=True)      # Convert spaces to single underscore
    .str.replace(r"_+", "_", regex=True)       # Collapse multiple underscores to one
)

In [41]:
df = df.dropna(subset=["registration_number"])

In [42]:
# Check for duplicate column names
duplicates = df.columns[df.columns.duplicated()]
print("Duplicate columns:", duplicates.tolist())
df = df.loc[:, ~df.columns.duplicated()]

Duplicate columns: []


In [43]:
df = df.drop(columns="source_file")
df = df.drop(columns="match_score")


In [44]:
# Get null ratios sorted
null_ratios = df.isnull().mean().sort_values(ascending=False)

# Convert to DataFrame with formatted percentage
summary_df = pd.DataFrame({
    "column": null_ratios.index,
    "null_ratio": (null_ratios * 100).map("{:.2f}%".format)
})

# Save to CSV
summary_df.to_csv("../../column_percentages/profile_null_ratio_summary.csv", index=False)
# summary_df.head()